<a href="https://colab.research.google.com/github/Raaj-4320/the-exam-paper-generator/blob/main/que_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.0/337.0 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.8 MB/s eta 0:00:00


In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
import json

In [ ]:
# Download NLTK data
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
def search_wikipedia(topic):
    search_url = f"https://en.wikipedia.org/w/api.php?action=query&list=search&srsearch={topic}&format=json"
    response = requests.get(search_url)
    search_results = response.json()
    return search_results['query']['search']

def clean_snippet(snippet):
    soup = BeautifulSoup(snippet, "html.parser")
    text = soup.get_text()
    return text

def preprocess_text(text):
    # Tokenize the text
    tokens = word_tokenize(text)

    # Convert to lowercase
    tokens = [word.lower() for word in tokens]

    # Remove stop words and non-alphabetic tokens
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word.isalpha() and word not in stop_words]

    return ' '.join(tokens)


topic = "List in python"
search_results = search_wikipedia(topic)

# Extract titles from the search results
titles = [result['title'] for result in search_results]

# Print related topics (titles)
all_topics = []
print("Related topics:")
for title in titles:
  all_topics.append(title)

print(all_topics)

Related topics:
['Python (programming language)', 'List of Python software', 'Reticulated python', 'Burmese python', 'Comparison of integrated development environments', 'Python syntax and semantics', 'Zen of Python', 'History of Python', 'Ball python', 'Python (genus)']


In [ ]:
import requests

def get_wikipedia_extract(title, length="short"):
    # Define the endpoint
    endpoint = "https://en.wikipedia.org/w/api.php"

    # Set the extract length parameters
    if length == "short":
        exchars = 1000  # Limit to approximately one paragraph
        params = {
            "action": "query",
            "format": "json",
            "titles": title,
            "prop": "extracts",
            "exchars": exchars,
            "explaintext": True
        }
    elif length == "medium":
        exsentences = 10  # Limit to approximately two paragraphs
        params = {
            "action": "query",
            "format": "json",
            "titles": title,
            "prop": "extracts",
            "exsentences": exsentences,
            "explaintext": True
        }
    else:
        params = {
            "action": "query",
            "format": "json",
            "titles": title,
            "prop": "extracts",
            "exintro": True,
            "explaintext": True
        }

    # Make the request to the Wikipedia API
    response = requests.get(endpoint, params=params)
    data = response.json()

    # Extract the page information
    pages = data['query']['pages']
    page = next(iter(pages.values()))  # Get the first (and only) page

    # Extract the extract text
    extract = page.get("extract", "No extract found.")

    return extract

# Dictionary to store the extracts
corpus = {}

# Iterate through each title and get a paragraph (full extract) for each
for title in titles:
    extract = get_wikipedia_extract(title, length="full")
    print(f"Title: {title}\nExtract:\n{extract}\n")
    corpus[title] = extract

# Now 'corpus' contains all the extracts with titles as keys
print("Corpus dictionary:", corpus)


Title: Python (programming language)
Extract:
Python is a high-level, general-purpose programming language. Its design philosophy emphasizes code readability with the use of significant indentation.
Python is dynamically typed and garbage-collected. It supports multiple programming paradigms, including structured (particularly procedural), object-oriented and functional programming. It is often described as a "batteries included" language due to its comprehensive standard library.
Guido van Rossum began working on Python in the late 1980s as a successor to the ABC programming language and first released it in 1991 as Python 0.9.0. Python 2.0 was released in 2000. Python 3.0, released in 2008, was a major revision not completely backward-compatible with earlier versions. Python 2.7.18, released in 2020, was the last release of Python 2.
Python consistently ranks as one of the most popular programming languages, and has gained widespread use in the machine learning community.



Title: L

In [ ]:
from openai import OpenAI

client = OpenAI(
    api_key=''
)

In [ ]:
completion = client.chat.completions.create(
    model='gpt-3.5-turbo',
    response_format={ "type": "json_object" },
    messages=[
        {'role':'system','content':'you are a good teacher who sets a good exam papers'},
        {'role': 'user','content':f'create exact 50 numbers multiple choice questions of this given {extract} with complexity easy, and mention all answers the with the questions with ABCD labels. Please do not explain the ansnwer, just return whatever is written in the options and the exact answer not more not less, give the answer with its respected label from ABCD and whatever written in that option of the answer. Give this output ins json format. Make sure to create 50 numbers of question'}
        ]
)

In [ ]:
print(completion.choices[0].message.content)

{
  "questions": [
    {
      "question": "1. How many python species are currently recognized as valid taxa?",
      "options": {
        "A": "10",
        "B": "5",
        "C": "15",
        "D": "20"
      },
      "answer": "A"
    },
    {
      "question": "2. Who proposed the name 'python' for non-venomous flecked snakes?",
      "options": {
        "A": "François Marie Daudin",
        "B": "Charles Darwin",
        "C": "Gregor Mendel",
        "D": "Isaac Newton"
      },
      "answer": "A"
    },
    {
      "question": "3. How many formerly considered python subspecies have been promoted?",
      "options": {
        "A": "3",
        "B": "2",
        "C": "5",
        "D": "10"
      },
      "answer": "A"
    },
    {
      "question": "4. What is the climate where pythons are native?",
      "options": {
        "A": "Tropics and subtropics of the Eastern Hemisphere",
        "B": "Arctic region",
        "C": "Desert region in the Western Hemisphere",
        "D":

In [ ]:
qa_data = completion.choices[0].message.content

In [ ]:

# Load JSON data into a Python dictionary
data = json.loads(qa_data)

# Create a dictionary with the desired format
cleaned_data = [
    {
        "question": item['question'],
        "choices": item['options'],
        "correct_answer": item['answer']
    }
    for item in data['questions']
]


In [ ]:
len(cleaned_data)

50

In [ ]:
for_loop = cleaned_data

In [ ]:
result = ""

for i in range(len(for_loop)):
    result += f"Question {i + 1}\n"
    result += cleaned_data[i]['question'] + "\n"

    # Print each choice with its label
    choices_dict = for_loop[i]['choices']
    for label, choice in choices_dict.items():
        result += f"{label} : {choice}\n"

    # Find the correct answer label
    correct_answer = for_loop[i]['correct_answer']
    result += f"Correct Answer: {correct_answer}\n\n"

# Optionally, print the result to see the output
print(result)

Question 1
When was YouTube TV announced?
A : February 28, 2017
B : March 15, 2020
C : January 10, 2018
D : April 5, 2016
Correct Answer: A

Question 2
How many television networks does YouTube TV offer content from?
A : 100
B : 50
C : 200
D : 75
Correct Answer: A

Question 3
In which markets does YouTube TV include affiliates of the Big Three broadcast networks?
A : Most markets
B : Only major cities
C : International markets
D : Only in rural areas
Correct Answer: A

Question 4
Which company operates YouTube TV?
A : YouTube
B : Amazon
C : Netflix
D : Hulu
Correct Answer: A

Question 5
How many subscribers did YouTube TV have as of February 9, 2024?
A : 8 million
B : 5 million
C : 10 million
D : 3 million
Correct Answer: A

Question 6
Where is YouTube TV available for streaming?
A : United States
B : Canada
C : Europe
D : Australia
Correct Answer: A

Question 7
What type of service is YouTube TV aimed at?
A : Cord cutters
B : Cable subscribers
C : Satellite TV users
D : Radio listener